# Vector Search Evaluation

1. Generation of **ground truth** dataset. 

✅ This step was compeleted in `01_evaluation_generation_ground_truth.ipynb` notebook.

  This can be done:
  * Manually by annotators / domain experts
  * Getting the data from users queries
  * Generate with LLM


Generally for one query, we might have multiple relevant documents, but for this use case, we have 1 relevant document(answer) for 1 query (user question).

The automatic generation of the dataset will be done as follows:
1. For every user query (question) LLM will be prompted to generate 5 similar questions
2. Apply vector search using our LLM-generated questions as queries to find relevant document in the knowledge base 
3. During the test phase we will evaluate our vector search to be able to detect relevant document for similar queries (aka. generated ones)

-------------
2. Evaluation of **Text Retrieval** techniques.

✅ Step compelted in `02_evaluation_text_retrieval.ipynb`

For every record in ground truth dataset we will:
  1. Execute query (perform text search in our vector database)
  2. Check if the retrieved results contain the answer assigned to the original query (from which we've generated our artificial queries)
  3. Perform metrics calculations
      * **Hit Rate** (or Recall)
      * **MRR (Mean Reciprocal** Rank)


-------------
3. Evaluation of **Vector Retrieval** techniques.

For every record in ground truth dataset we will:
  1. Execute query (perform vector search in our vector database)
  2. Check if the retrieved results contain the answer assigned to the original query (from which we've generated our artificial queries)
  3. Perform metrics calculations
      * **Hit Rate** (or Recall)
      * **MRR (Mean Reciprocal** Rank)

In [5]:
from sentence_transformers import SentenceTransformer

In [2]:
import json

with open('documents-with-ids.json', 'rt') as f_in:
    documents = json.load(f_in)

In [3]:
model_name = 'multi-qa-MiniLM-L6-cos-v1'
model = SentenceTransformer(model_name)

In [4]:
# embedding size
v = model.encode('I just discovered the course. Can I still join?')
len(v)

384

## 1. Indexing our KnowledgeBase data into ElasticSearch

### 1.1 Setup ElasticSearch connection

Running ES with Docker
```
docker run -it \
    --rm \
    --name elasticsearch \
    -p 9200:9200 \
    -p 9300:9300 \
    -e "discovery.type=single-node" \
    -e "xpack.security.enabled=false" \
    docker.elastic.co/elasticsearch/elasticsearch:8.4.3
```

In [9]:
from elasticsearch import Elasticsearch

In [10]:
!curl localhost:9200

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


{
  "name" : "675ce062f461",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "4gR5tzWATWyRv8KHoGRj7Q",
  "version" : {
    "number" : "8.4.3",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "42f05b9372a9a4a470db3b52817899b99a76ee73",
    "build_date" : "2022-10-04T07:17:24.662462378Z",
    "build_snapshot" : false,
    "lucene_version" : "9.3.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}


In [11]:
es_client = Elasticsearch('http://localhost:9200')
es_client.info()

ObjectApiResponse({'name': '675ce062f461', 'cluster_name': 'docker-cluster', 'cluster_uuid': '4gR5tzWATWyRv8KHoGRj7Q', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [12]:
health = es_client.cluster.health()
print(health)

{'cluster_name': 'docker-cluster', 'status': 'green', 'timed_out': False, 'number_of_nodes': 1, 'number_of_data_nodes': 1, 'active_primary_shards': 2, 'active_shards': 2, 'relocating_shards': 0, 'initializing_shards': 0, 'unassigned_shards': 0, 'delayed_unassigned_shards': 0, 'number_of_pending_tasks': 0, 'number_of_in_flight_fetch': 0, 'task_max_waiting_in_queue_millis': 0, 'active_shards_percent_as_number': 100.0}


In [13]:
# Adjust high and low watermarks temporarily
es_client.cluster.put_settings(body={
    "transient": {
        "cluster.routing.allocation.disk.watermark.low": "85%",
        "cluster.routing.allocation.disk.watermark.high": "95%",
        "cluster.routing.allocation.disk.watermark.flood_stage": "98%"
    }
})

ObjectApiResponse({'acknowledged': True, 'persistent': {}, 'transient': {'cluster': {'routing': {'allocation': {'disk': {'watermark': {'low': '85%', 'flood_stage': '98%', 'high': '95%'}}}}}}})

Using OpenAI API for generation task.

### 1.2 Indexing data

Building embeddings for question, answer and question+answer

In [14]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch('http://localhost:9200')

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "id": {"type": "keyword"},
            "question_vector": {
                "type": "dense_vector",
                "dims": 384,
                "index": True,
                "similarity": "cosine"
            },
            "text_vector": {
                "type": "dense_vector",
                "dims": 384,
                "index": True,
                "similarity": "cosine"
            },
            "question_text_vector": {
                "type": "dense_vector",
                "dims": 384,
                "index": True,
                "similarity": "cosine"
            },
        }
    }
}

index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [15]:
from tqdm.auto import tqdm

In [16]:
for doc in tqdm(documents):
    question = doc['question']
    text = doc['text']
    qt = question + ' ' + text

    doc['question_vector'] = model.encode(question)
    doc['text_vector'] = model.encode(text)
    doc['question_text_vector'] = model.encode(qt)

100%|██████████| 948/948 [01:15<00:00, 12.53it/s]


In [17]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|██████████| 948/948 [00:09<00:00, 103.72it/s]


### 1.2 Construct the search query (Semantic Search)

In [18]:
query = 'I juste discovered the course. Can  I still join?'

In [19]:
query_vector = model.encode(query)

In [20]:
search_query = {
    "field": "question_vector",
    "query_vector": query_vector,
    "k": 5,
    "num_candidates": 10000,
}

In [21]:
es_results = es_client.search(index=index_name,
                       knn=search_query,
                       # output parameter
                       source=["text", "section", "question", "course", "id"])


In [22]:
es_results

ObjectApiResponse({'took': 2061, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 5, 'relation': 'eq'}, 'max_score': 0.94541216, 'hits': [{'_index': 'course-questions', '_id': 'LEFiQpEBejrObOfxB6rc', '_score': 0.94541216, '_source': {'question': 'The course has already started. Can I still join it?', 'course': 'machine-learning-zoomcamp', 'section': 'General course-related questions', 'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.', 'id': 'ee58a693'}}, {'_index': 'course-questions', '_id': 'bUFhQpEBejrObOfx9agu', '_score': 0.8923334, '_source': {'question': 'Course - Can I still join the c

In [23]:
results_docs =[]

for hit in es_results["hits"]["hits"]:
    results_docs.append(hit["_source"])

In [26]:
results_docs

[{'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp',
  'section': 'General course-related questions',
  'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'id': 'ee58a693'},
 {'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp',
  'section': 'General course-related questions',
  'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'id': '7842b56a'},
 {'question': 'Course

Our search query provides output for different courses.
In case we want to be more specific we will modify our searech query.

In [27]:
def elastic_search_knn(field, vector, course):
    knn = {
        "field": field,
        "query_vector": vector,
        "k": 5,
        "num_candidates": 10000,
        "filter": {
            "term": {
                "course": course
            }
        }
    }

    search_query = {
        "knn": knn,
        "_source": ["text", "section", "question", "course", "id"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )

    result_docs = []

    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [28]:
elastic_search_knn("question_vector", query_vector, "data-engineering-zoomcamp")

[{'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp',
  'section': 'General course-related questions',
  'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'id': '7842b56a'},
 {'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp',
  'section': 'General course-related questions',
  'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'id': 'a482086d'},
 {'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp',
  'section': '

## 2. Evaluation 

Now we can iterate over our ground truth dataset(synthetic) to get the reponse back.
And check if the output are in the results.

In [29]:
import pandas as pd

In [30]:
df_ground_truth = pd.read_csv('ground-truth-data.csv')

In [31]:
ground_truth = df_ground_truth.to_dict(orient='records')

In [32]:
ground_truth[0]

{'question': 'When does the course begin?',
 'course': 'data-engineering-zoomcamp',
 'document': 'c02e79ef'}

In [33]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

In [34]:
def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)

In [35]:
def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['document']
        results = search_function(q)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

### 2.1 Evaluating search on question vector

In [36]:
def question_vector_knn(q):
    """ Performs a vector search of our question"""
    question = q['question']
    course = q['course']

    v_q = model.encode(question)

    return elastic_search_knn('question_vector', v_q, course)

In [37]:
evaluate(ground_truth, question_vector_knn)

100%|██████████| 4627/4627 [01:36<00:00, 47.81it/s]


{'hit_rate': 0.773071104387292, 'mrr': 0.6666810748505158}

### 2.2 Evaluating search on question+text vector

In [38]:
def question_text_vector_knn(q):
    """ Performs a vector search of our question+text"""
    question = q['question']
    course = q['course']

    v_q = model.encode(question)

    return elastic_search_knn('question_text_vector', v_q, course)


In [39]:
evaluate(ground_truth, question_text_vector_knn)

100%|██████████| 4627/4627 [01:28<00:00, 52.04it/s]


{'hit_rate': 0.9172249837907932, 'mrr': 0.824306606152295}

### 2.3 Evaluating search on text(answer) vector

In [40]:
def text_vector_knn(q):
    """ Performs a vector search of our text"""

    question = q['question']
    course = q['course']

    v_q = model.encode(question)

    return elastic_search_knn('text_vector', v_q, course)

In [41]:
evaluate(ground_truth, text_vector_knn)

100%|██████████| 4627/4627 [01:21<00:00, 56.59it/s]


{'hit_rate': 0.8286146531229739, 'mrr': 0.7062315395144454}

### 2.4 Evaluating search on all 3 vectors

For this we will update our search query in order to combine 3 vectors

In [42]:
def elastic_search_knn_combined(vector, course):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": [
                    {
                        "script_score": {
                            "query": {
                                "term": {
                                    "course": course
                                }
                            },
                            "script": {
                                "source": """
                                    cosineSimilarity(params.query_vector, 'question_vector') +
                                    cosineSimilarity(params.query_vector, 'text_vector') +
                                    cosineSimilarity(params.query_vector, 'question_text_vector') +
                                    1
                                """,
                                "params": {
                                    "query_vector": vector
                                }
                            }
                        }
                    }
                ],
                "filter": {
                    "term": {
                        "course": course
                    }
                }
            }
        },
        "_source": ["text", "section", "question", "course", "id"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )

    result_docs = []

    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [43]:
def vector_combined_knn(q):
    question = q['question']
    course = q['course']

    v_q = model.encode(question)

    return elastic_search_knn_combined(v_q, course)

evaluate(ground_truth, vector_combined_knn)

100%|██████████| 4627/4627 [01:48<00:00, 42.53it/s]


{'hit_rate': 0.9023125135076724, 'mrr': 0.804480945176861}

The best results are provided by our vector search performed on question+answer. Combining 3 vectors search seems to be not useful and not brinign any advantages.